# INSTALLATION

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
##Clone github repository##
!git clone https://github.com/liraserge26/TFG_Mask_RCNN.git
    
!pip uninstall keras-nightly
!pip uninstall -y tensorflow
!pip uninstall -y imgaug

!pip install imgaug==0.2.6
!pip install -U scikit-image==0.16.2
!pip install h5py==2.10.0
!pip install tensorflow==1.15
!pip install tensorflow-gpu==1.15.0
!pip install keras==2.1.6

%cd TFG_Mask_RCNN

!pip3 install -r requirements.txt
!python3 setup.py install

!git clone https://github.com/cocodataset/cocoapi.git
%cd cocoapi/PythonAPI
!make
%cd ../../

## CONFIGURATIONS



In [ ]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import json
import imgaug

ROOT_DIR = os.path.abspath("/content/drive/MyDrive/TFG/Mask_RCNN/Data/PCB_Holes/Experiment_1")

sys.path.append(ROOT_DIR)  

from mrcnn.config import Config

from mrcnn import utils

import mrcnn.model as modellib

from mrcnn import visualize

from mrcnn.model import log

%matplotlib inline 

MODEL_DIR = os.path.join(ROOT_DIR, "logs")


COCO_MODEL_PATH = os.path.join(MODEL_DIR, "mask_rcnn_coco.h5")

if not os.path.exists(COCO_MODEL_PATH) or not os.path.exists(COCO_MODEL_PATH_2):
  %cd Data/PCB_Holes/Experiment_1/logs
  utils.download_trained_weights(os.path.join(os.path.abspath("../"),"mask_rcnn_coco.h5"))

In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

Holes_Verification_DATA_DIR = '/content/drive/MyDrive/TFG/Mask_RCNN/Data/'

DATASET_DIR = os.path.join(Holes_Verification_DATA_DIR,'PCB_Holes')   

DATASET_TRAIN_DIR = os.path.join(DATASET_DIR, 'LabelImg_JSON/train')
DATASET_VAL_DIR = os.path.join(DATASET_DIR, 'LabelImg_JSON/val')
DATASET_TEST_DIR = os.path.join(DATASET_DIR, 'LabelImg_JSON/test')

config = ConfigProto()

config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
class PCB_Data(Config):
    """Configuration for training on the Raps  dataset.
    Derives from the base Config class and overrides some values.
    """
    NAME = "PCB_Data"

    IMAGES_PER_GPU = 1
  

    NUM_CLASSES = 1 + 2 # Background + Holes_Ok + Hole_Not_Ok 


    STEPS_PER_EPOCH = 86 


    DETECTION_MIN_CONFIDENCE = 0.1
    DETECTION_NMS_THRESHOLD = 0.1
    DETECTION_MAX_INSTANCES = 500


    USE_MINI_MASK = True

    IMAGE_SHAPE = [1024,1024,3]
    BATCH_SIZE =4

    LEARNING_RATE=0.005

	
config = PCB_Data()
config.display()

In [ ]:
from PIL import Image,ImageDraw

class PCB_dataDataset(utils.Dataset):
    def load_dataset(self, dataset_dir):
        self.add_class('dataset', 1, 'hole_ok')

        self.add_class('dataset', 2, 'hole_not_ok')
        for i, filename in enumerate(os.listdir(dataset_dir)):
            annotation_file = os.path.join(dataset_dir, filename.replace('.jpg', '.json'))
            if '.jpg' in filename and os.path.isfile(annotation_file):
                    self.add_image('dataset', 
                               image_id=i, 
                               path=os.path.join(dataset_dir, filename),
                               annotation=annotation_file)
    
    
    def extract_masks(self, filename, image_id):
        print(image_id)
        image = self.load_image(image_id)
        json_file = os.path.join(filename)
        with open(json_file) as f:
            img_anns = json.load(f)

        n_masks = 0
        for shape in img_anns[0]['annotations']:
          if shape['label']=='hole_ok' or 'hole_not_ok':
            n_masks+=1
       
        masks = np.zeros([image.shape[0], image.shape[1], n_masks], dtype='uint8')
        classes = []
        i=0
        for shape in img_anns[0]['annotations']:
          area = self.shape_area(shape['coordinates'])
          mask = np.zeros([image.shape[0], image.shape[1]], dtype='uint8')
          cv2.fillPoly(mask, area, 255)
          masks[:, :, i] = mask                                          
          classes.append(self.class_names.index(shape['label']))
          i+=1
        return masks, classes
 
    def load_mask(self, image_id):
        info = self.image_info[image_id]
        path = info['annotation']
        masks, classes = self.extract_masks(path, image_id)
        return masks, np.asarray(classes, dtype='int32')
    
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path']

    def shape_area(self, bbox):
        x=(bbox['x'])
        y=(bbox['y'])
        width=(bbox['width'])
        height=(bbox['height'])
        x0=(x-(width/2))
        y0=(y-(height/2))
        xf=(x+(width/2))
        yf=(y+(height/2))
        p1=([int(x0), int(yf)])
        p2=([int(xf), int(yf)])
        p3=([int(xf), int(y0)])
        p4=([int(x0), int(y0)])
        area = np.array([[p1, p2, p3, p4]], dtype=np.int32)
        return area

In [ ]:
dataset_train = PCB_dataDataset()
dataset_train.load_dataset(DATASET_TRAIN_DIR)
dataset_train.prepare()
print('Training:')

print("Image Count: {}".format(len(dataset_train.image_ids)))
print("Class Count: {}".format(dataset_train.num_classes))
for i, info in enumerate(dataset_train.class_info):
    print("{:3}. {:50}".format(i, info['name']))

 
dataset_val = PCB_dataDataset()
dataset_val.load_dataset(DATASET_VAL_DIR)
dataset_val.prepare()
print('Validation:')

print("Image Count: {}".format(len(dataset_val.image_ids)))
print("Class Count: {}".format(dataset_val.num_classes))
for i, info in enumerate(dataset_val.class_info):
    print("{:3}. {:50}".format(i, info['name']))

dataset_test = PCB_dataDataset()
dataset_test.load_dataset(DATASET_TEST_DIR)
dataset_test.prepare()
print('Test:')

print("Image Count: {}".format(len(dataset_test.image_ids)))
print("Class Count: {}".format(dataset_test.num_classes))
for i, info in enumerate(dataset_test.class_info):
    print("{:3}. {:50}".format(i, info['name']))

In [ ]:
##Choose 5 random images from training dataset and visualize masks##

image_ids = np.random.choice(dataset_train.image_ids, 5)
for image_id in image_ids:
  image = dataset_train.load_image(image_id)
  mask, class_ids = dataset_train.load_mask(image_id)
  visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)

## Create Model and Load Trained Weights

In [ ]:
# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config, model_dir=MODEL_DIR)

In [ ]:
##init_with=="coco" if we train the model with random weights
##init_with=="lats" if we train the model with the last calculated weights

init_with = "last"  

if init_with == "coco":
    model.load_weights(COCO_MODEL_PATH, by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    model.load_weights(model.find_last(), by_name=True)

## Train the model

In [ ]:
##with layers="heads" we train the last layers for training specific features.

PCB_augmentation = imgaug.augmenters.Sometimes(0.5,
    [imgaug.augmenters.geometric.Affine(rotate=(-360,360))])
    
    
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE, 
            epochs=130,
            layers='heads',augmentation = PCB_augmentation)

In [ ]:
##with layers="all" we train all the layers.

model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE,
            epochs=120, 
            layers="all", augmentation = PCB_augmentation)

## Inference

In [ ]:
##Create the model in inference mode
class InferenceConfig(PCB_Data):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()

model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

model_path = model.find_last()


print("Loading weights from ", model_path)

model.load_weights(model_path, by_name=True)


In [ ]:
##Choose a random image from test dataset and visualize the mask
image_id = np.random.choice(dataset_test.image_ids)
original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset_test, inference_config, 
                           image_id, use_mini_mask=False)

log("original_image", original_image)
log("image_meta", image_meta)
log("gt_class_id", gt_class_id)
log("gt_bbox", gt_bbox)
log("gt_mask", gt_mask)

visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id,
                            dataset_train.class_names, figsize=(12, 12))

print(gt_class_id.shape)

In [ ]:
##Save predicted mask from the random image and visualize it
results = model.detect([original_image], verbose=1)

r = results[0]
visualize.display_instances(original_image, r['rois'], r['masks'], r['class_ids'], 
                            dataset_test.class_names, r['scores'])

## Results with Confusion matrix

In [ ]:
##Save all predicted masks from test dataset and visualize the results using confusion matrix algorithm

import pandas as pd
import numpy as np
import os 
from mrcnn import utils_cm
from mrcnn import utils


gt_tot = np.array([])
pred_tot = np.array([])

mAP_ = []


for image_id in dataset_test.image_ids:
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_test, config, image_id, use_mini_mask=False)
    info = dataset_test.image_info[image_id]


    results = model.detect([image], verbose=1)
    r = results[0]

    gt, pred = utils_cm.gt_pred_lists(gt_class_id, gt_bbox, r['class_ids'], r['rois'])
    gt_tot = np.append(gt_tot, gt)
    pred_tot = np.append(pred_tot, pred)
    

    AP_, precision_, recall_, overlap_ = utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                                          r['rois'], r['class_ids'], r['scores'], r['masks'])

    print("the actual len of the gt vect is : ", len(gt_tot))
    print("the actual len of the pred vect is : ", len(pred_tot))
    
    mAP_.append(AP_)
    print("Average precision of this image : ",AP_)
    print("The actual mean average precision for the whole images (matterport methode) ", sum(mAP_)/len(mAP_))
  
gt_tot=gt_tot.astype(int)
pred_tot=pred_tot.astype(int)

save_dir = os.path.join(MODEL_DIR, "Confusion_Matrix")
gt_pred_tot_json = {"gt_tot" : gt_tot, "pred_tot" : pred_tot}
df = pd.DataFrame(gt_pred_tot_json)
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
df.to_json(os.path.join(save_dir,"gt_pred_test.json"))

gt_class_id = np.array([1,2,1,2]) #[hole_ok, hole_not_ok, hole_ok, hole_not_ok]

print("ground truth list : ",gt_tot)
print("predicted list : ",pred_tot)
print(gt_tot)
print(pred_tot)

tp,fp,fn=utils_cm.plot_confusion_matrix_from_data(gt_tot,pred_tot,dataset_test.class_names,fz=18, figsize=(20,20), lw=0.5)